In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/Representation\ Discrimination

 ethnic_dataset_cleaned.joblib
 Representation_Discrimination_10187.ipynb
'Representation_Discrimination - Word Embedding Based Approach.ipynb'


In [4]:
import joblib
cleaned_10k_articles = joblib.load("/content/drive/MyDrive/Representation Discrimination/ethnic_dataset_cleaned.joblib")
cleaned_10k_articles[:10]

['আন্তর্জাতিক আদিবাসী দিবস উপলক্ষে বুধবার বিভিন্ন স্থানে সভা-সমাবেশ ও রযালি অনুষ্ঠিত হয়েছে। উপজাতিদের বিভিন্ন সংগঠন এসব কর্মসূচির আয়োজন করে। যুগান্তর ব্যুরো ও প্রতিনিধিদের পাঠানো খবর-সিলেট ব্যুরো : রযালিতে সিলেট প্রেস ক্লাবের সভাপতি ইকরামুল কবির, ইমজার সভাপতি আল আজাদসহ সিলেটের সাংস্কৃতিক অঙ্গনের ব্যক্তি ও উপজাতি জনগোষ্ঠীর লোকজন ছাড়াও বিভিন্ন শ্রেণী-পেশার মানুষ অংশগ্রহণ করেন। এছাড়া দিবসটি উপলক্ষে শাহজালাল বিজ্ঞান ও প্রযুক্তি বিশ্ববিদ্যালয়ে আলোচনা সভা অনুষ্ঠিত হয়েছে। ‘অ্যাসোসিয়েশন অব ইন্ডিজেনাস স্টুডেন্টস, সাস্ট’ ও সমাজকর্ম বিভাগ আলাদা আলাদাভাবে দিবসটি উপলক্ষে রযালি, সেমিনার ও আলোচনা সভার আয়োজন করে।ময়মনসিংহ ব্যুরো : অনুষ্ঠানের উদ্বোধন করেন জেলা পরিষদের চেয়ারম্যান অধ্যাপক ইউসুফ খান পাঠান। বক্তব্য রাখেন হিন্দু বৌদ্ধ খ্রিস্টান ঐক্য পরিষদের সভাপতি বিকাশ রায়, সিপিবি সভাপতি এমদাদুল হক মিল্লাত, মহিলা পরিষদের সভানেত্রী ফেরদৌস আরা মাহমুদা হেলেন, হিল্লুুল নকরেক, অরণ্য-ই চিরান, স্বর্ণকান্ত হাজং সাংবাদিক শরীফুজ্জামান টিটু প্রমুখ।শেরপুর : বাগাছাস সভাপতি রুনু নকরেটের সভাপতিত্বে অনুষ্ঠিত আলোচনা সভায় প্

# Bangla Topic Modeling

In [5]:
!wget https://github.com/aljubaer/Topic-Modeling-in-Bengali/blob/master/code/stop_words.txt

--2024-04-28 17:10:28--  https://github.com/aljubaer/Topic-Modeling-in-Bengali/blob/master/code/stop_words.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘stop_words.txt’

stop_words.txt          [ <=>                ] 424.27K  --.-KB/s    in 0.06s   

2024-04-28 17:10:28 (6.44 MB/s) - ‘stop_words.txt’ saved [434454]



In [6]:
!pip3 install sbnltk

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 826.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cuso

In [7]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import nltk
import json
import os

nltk.download('punkt')

def valid_bengali_letters(char):
    return ord(char) >= 2433 and ord(char) <= 2543

def get_replacement(char):
    if valid_bengali_letters(char):
        return char
    newlines = [10, 2404, 2405, 2551, 9576]
    if ord(char) in newlines:
        return ' '
    return ' ';

def get_valid_lines(line):
    copy_line = ''
    for letter in line:
        copy_line += get_replacement(letter)
    return copy_line

def sent_to_words(sentences):
    for sentence in sentences:
        yield(nltk.word_tokenize(get_valid_lines(sentence)))  # deacc=True removes punctuations

from sbnltk.Stemmer import stemmerOP
bn_stemmer=stemmerOP()
def stemming_data(content):
    for i, word in enumerate(content):
        content[i] = bn_stemmer.stemWord(word)
    return content
def stemming_data_list(data_list):
    for i, sdata in enumerate(data_list):
        data_list[i] = stemming_data(sdata)
    return data_list

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Downloading...
From: https://drive.google.com/uc?id=142XvJg9xdpgzuYD31Y4pm-ZVdMaWmtuq
To: /usr/local/lib/python3.10/dist-packages/sbnltk/dataset/download_link.txt
100%|██████████| 1.66k/1.66k [00:00<00:00, 809kB/s]
Downloading...
From: https://drive.google.com/uc?id=1UIR2arP_6Fm0MqJVbcG2Q9iLCnjQriDB
To: /usr/local/lib/python3.10/dist-packages/sbnltk/dataset/bangla_word_list.txt
100%|██████████| 15.4M/15.4M [00:00<00:00, 102MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1e4x5FQrdGyEfBBzQ_DOJ9wBxZZ1F7bx7
To: /usr/local/lib/python3.10/dist-packages/sbnltk/dataset/stopword_list.txt
100%|██████████| 4.77k/4.77k [00:00<00:00, 3.31MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ohRYMGAeq4RZTa3W-7DXEsaUCaVNIxgO
To: /usr/local/lib/python3.10/dist-packages/sbnltk/dataset/rootword_list.txt
100%|██████████| 2.00M/2.00M [00:00<00:00, 121MB/s]
Downloading...
From: https://

In [8]:
stopwords_file = open('stop_words.txt', "r+", encoding = 'utf-8')
all_stopwords = stopwords_file.read()
stopwords_ready = [word.strip() for word in all_stopwords.split()]
def convertToDataFrame(data_json):
    df = pd.DataFrame(data_json)
    print('DataFrame shape' + str(df.shape))
    return df

def remove_stopwords(content):
    without_stopwords = []
    for word in content:
        if word not in stopwords_ready and len(word) > 5:
            without_stopwords.append(word)
    return without_stopwords
def remove_stopwords_list(data_list):
    data_without_stopwords_list = []
    for content in data_list:
        data_without_stopwords_list.append(remove_stopwords(content))
    return data_without_stopwords_list
def runLda(data_ready, num_topics = 10, iterations = 1000, alpha='auto'):
    # Create Dictionary
    id2word = corpora.Dictionary(data_ready)

    # Create Corpus: Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in data_ready]

    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=num_topics,
                                               random_state=100,
                                               update_every=1,
                                               chunksize=30,
                                               passes=30,
                                               alpha=alpha,
                                               iterations=iterations,
                                               per_word_topics=True)
    return lda_model
def ldaOutputProducer(lda_model):
    x = (lda_model.show_topics(num_topics=20, num_words=40,formatted=False))
    topics_words = [(tp[0], [wd[0] for wd in tp[1]], [wd[1] for wd in tp[1]]) for tp in x]
    output_json_list = []
    for topic,words,conts in topics_words:
        topic_json = {}
        topic_content = {}
        topic_content["words"] = words
        topic_content["conts"] = conts
        #topic_json[str(topic)] = topic_content
        output_json_list.append(topic_content)
    out_df = convertToDataFrame(output_json_list)
    out_df.to_json(r'topic_dist.txt')

In [9]:
data_tokenized_list = list(sent_to_words(cleaned_10k_articles))
data_without_stopwords_list = remove_stopwords_list(data_tokenized_list)
data_stemmed_list = stemming_data_list(data_without_stopwords_list)
lda_model = runLda(data_stemmed_list, num_topics = 10, iterations = 1000)
for topic_id, topic_words in lda_model.print_topics():
    print(f"Topic ID: {topic_id}, Words: {topic_words}")
ldaOutputProducer(lda_model)

Topic ID: 0, Words: 0.022*"পার্বত্য" + 0.021*"প্রধানমন্ত্রী" + 0.021*"বাংলাদেশ" + 0.018*"চট্টগ্রাম" + 0.017*"চাকমা" + 0.013*"ত্রিপুরা" + 0.013*"খাগড়াছড়ি" + 0.012*"ইউপিডিএফ" + 0.012*"বিএনপি" + 0.010*"রাঙ্গামাট"
Topic ID: 1, Words: 0.054*"শিক্ষা" + 0.045*"শিক্ষক" + 0.041*"সরকারি" + 0.039*"প্রাথমিক" + 0.026*"চাকরিতে" + 0.022*"সন্তান" + 0.019*"প্রশিক্ষণ" + 0.017*"বিদ্যাল" + 0.015*"শিশু" + 0.015*"সংস্ক"
Topic ID: 2, Words: 0.074*"নির্বাচন" + 0.056*"প্রার্থী" + 0.056*"স্বাস্থ্য" + 0.045*"মুক্তিযোদ্ধা" + 0.024*"মোহাম্মদ" + 0.021*"পরিকল্পনা" + 0.021*"নির্বাচিত" + 0.017*"জানা" + 0.014*"পেয়" + 0.014*"শক্তিমান"
Topic ID: 3, Words: 0.031*"উপজেলা" + 0.025*"সম্পাদক" + 0.018*"উপজেল" + 0.017*"চেয়ারম্যান" + 0.017*"আওয়ামী" + 0.017*"সভাপতি" + 0.015*"ইউনিয়ন" + 0.015*"সাধারণ" + 0.015*"শিক্ষার্থী" + 0.012*"চৌধুরী"
Topic ID: 4, Words: 0.049*"আক্তার" + 0.039*"বৃহস্পতিব" + 0.018*"চন্দ্" + 0.015*"হাইকোর্ট" + 0.015*"ওবায়দুল" + 0.014*"মিয়ানমারে" + 0.013*"মোবাইল" + 0.013*"উদ্দিন" + 0.013*"মোটরসাইকেল" + 0.012*"দুর্